In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import time
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split, Subset
from torchvision.datasets import ImageFolder
from torchvision.models import resnet50
import torch.nn as nn
import torch.optim as optim
from torch.cuda.amp import GradScaler, autocast
from torch.utils.data import WeightedRandomSampler

In [3]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def load_data(data_dir, batch_size, num_workers, pin_memory):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize images
    ])

    # Load the dataset from folders
    full_dataset = ImageFolder(root=data_dir, transform=transform)

    # Calculate class weights and sample weights
    class_counts = torch.zeros(len(full_dataset.classes))
    for _, index in full_dataset.samples:
        class_counts[index] += 1

    class_weights = 1.0 / (class_counts + 1e-6)  # Add a small constant to avoid division by zero
    sample_weights = torch.tensor([class_weights[i] for i in full_dataset.targets])

    # Split into training and testing sets
    train_size = int(0.8 * len(full_dataset))
    test_size = len(full_dataset) - train_size
    train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

    # WeightedRandomSampler for the training set
    train_sampler = WeightedRandomSampler(weights=sample_weights[train_dataset.indices], num_samples=len(train_dataset), replacement=True)

    # DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=train_sampler, num_workers=num_workers, pin_memory=pin_memory)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=pin_memory)

    return train_loader, test_loader


In [4]:
# Model definition
def define_model():
    model = resnet50(pretrained=True)
    num_features = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Linear(num_features, 512),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.BatchNorm1d(512),
        nn.Linear(512, 50)  # 50 classes
    )
    return model.to(device)




In [5]:
# Training the model
def train_model(model, train_loader, epochs, learning_rate):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    scaler = GradScaler()

    for epoch in range(epochs):
        model.train()
        epoch_start_time = time.time()
        for i, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)

            with autocast():
                outputs = model(images)
                loss = criterion(outputs, labels)

            optimizer.zero_grad()
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            if (i + 1) % 20 == 0:
                print(f'Epoch [{epoch+1}/{epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

        epoch_end_time = time.time()
        epoch_duration = epoch_end_time - epoch_start_time
        print(f'Epoch [{epoch+1}/{epochs}] completed in {epoch_duration:.2f} seconds')


In [6]:
# Model evaluation
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Accuracy of the model on the test images: {accuracy:.2f}%')

In [7]:
# Main function
def main():
    data_dir = '/content/drive/MyDrive/JPEGImages'
    batch_size = 64
    num_workers = 8
    pin_memory = True
    epochs = 10
    learning_rate = 0.001

    train_loader, test_loader = load_data(data_dir, batch_size, num_workers, pin_memory)
    model = define_model()
    train_model(model, train_loader, epochs, learning_rate)
    evaluate_model(model, test_loader)

if __name__ == '__main__':
    main()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 162MB/s]


Epoch [1/10], Step [20/129], Loss: 3.0583
Epoch [1/10], Step [40/129], Loss: 2.5757
Epoch [1/10], Step [60/129], Loss: 2.1621
Epoch [1/10], Step [80/129], Loss: 1.7886
Epoch [1/10], Step [100/129], Loss: 1.9454
Epoch [1/10], Step [120/129], Loss: 1.7703
Epoch [1/10] completed in 249.43 seconds
Epoch [2/10], Step [20/129], Loss: 1.7760
Epoch [2/10], Step [40/129], Loss: 1.9114
Epoch [2/10], Step [60/129], Loss: 1.3262
Epoch [2/10], Step [80/129], Loss: 1.5620
Epoch [2/10], Step [100/129], Loss: 1.4433
Epoch [2/10], Step [120/129], Loss: 1.1647
Epoch [2/10] completed in 102.83 seconds
Epoch [3/10], Step [20/129], Loss: 1.1011
Epoch [3/10], Step [40/129], Loss: 1.1316
Epoch [3/10], Step [60/129], Loss: 0.9999
Epoch [3/10], Step [80/129], Loss: 1.0285
Epoch [3/10], Step [100/129], Loss: 1.1363
Epoch [3/10], Step [120/129], Loss: 1.1289
Epoch [3/10] completed in 59.93 seconds
Epoch [4/10], Step [20/129], Loss: 0.8005
Epoch [4/10], Step [40/129], Loss: 1.2245
Epoch [4/10], Step [60/129], Los